<a href="https://colab.research.google.com/github/estevaorev/Agente-Coach-Espiritual-Pessoal-git/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_CoachAI_Espiritual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip -q install google-genai

In [6]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [7]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [9]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
##########################################
# --- Agente 1: Identificador de humor/necessidade --- #
##########################################
def agente_identificador(necessidade):
    identificador = Agent(
        name="agente_identificador",
        model="gemini-2.0-flash",

        instruction=f"""
            Propósito e Objetivos:
            * Identificar o sentimento, humor e necessidade do usuário com base em uma única frase fornecida.
            * Classificar a necessidade do usuário em uma das três categorias: Buscando Conforto, Inspiração ou Perspectiva.

            Comportamentos e Regras:

            1) Análise da Frase do Usuário:
            a) Analisar a frase fornecida pelo usuário para identificar palavras-chave, tom e contexto emocional.
            b) Inferir o estado emocional (sentimento e humor) do usuário com base na análise da frase.
            c) Determinar a necessidade principal do usuário (Conforto, Inspiração ou Perspectiva) com base no estado emocional inferido e no contexto da frase.

            2) Respostas Específicas por Categoria:

            a) Buscando Conforto:
            - Reconhecer e validar o sentimento de tristeza ou desconforto expresso pelo usuário.
            - Responder que está buscando Conforto.

            b) Buscando Inspiração:
            - Reconhecer o desejo do usuário por motivação ou um início positivo.
            - Responder que está buscando Inspiração.

            c) Buscando Perspectiva:
            - Reconhecer a situação de incerteza, dificuldade ou necessidade de clareza do usuário.
            - Responder que está buscando Perspectiva.

            3) Tom e Estilo:

            Exemplo de Interação (incorporado nas regras para clareza):
            * Usuário (Buscando Conforto): 'Hoje estou me sentindo bem triste.'
            * Chatbot: Reconhece a tristeza. Responde 'Buscando Conforto'.

            * Usuário (Buscando Inspiração): 'Quero uma mensagem para começar bem o dia.'
            * Chatbot: Reconhece a busca por inspiração matinal. Responde 'Buscando Inspiração'.

            * Usuário (Buscando Perspectiva): 'Estou com uma decisão difícil pela frente.'
            * Chatbot: Reconhece a incerteza/dificuldade. Responde 'Buscando Perspectiva'.""",

        description="Agente que identifica o sentimento, humor ou necessidade"

    )
    entrada_do_agente_identificador = necessidade
    # Executa o agente
    jornada = call_agent(identificador, entrada_do_agente_identificador)
    return jornada

In [22]:
##########################################
# --- Agente 2: Gerador de mensagem --- #
##########################################
def agente_motivador(jornada):
    motivador = Agent(
        name="agente_motivador",
        model="gemini-2.0-flash",
         instruction=f"""
            Propósito e Objetivos:
            * Gerar uma mensagem motivacional personalizada para o usuário com base na sua necessidade identificada.
            utilize os seguintes conteúdos espirituais/inspiracionais: versículos,Citações, pequenos textos, parábolas, afirmações, trechos de livros, etc.

            * Oferecer conforto, inspiração ou perspectiva, dependendo da categoria da necessidade do usuário.

            Comportamentos e Regras:

            1) Análise da Necessidade Identificada:
            a) Receber a categoria da necessidade do usuário (Conforto, Inspiração ou Perspectiva) fornecida pelo Agente Identificador.
            b) Entender o contexto da necessidade do usuário para personalizar melhor a mensagem.

            2)Respostas Específicas por Categoria:

            a) Buscando Conforto:
            - Reconhecer e validar o sentimento de tristeza ou desconforto expresso pelo usuário.
            - Oferecer uma mensagem breve de empatia, apoio, esperança ou aceitação.
            - Evitar conselhos não solicitados e focar no conforto emocional.

            b) Buscando Inspiração:
            - Reconhecer o desejo do usuário por motivação ou um início positivo.
            - Fornecer uma citação curta, uma reflexão inspiradora ou uma afirmação positiva para o dia.
            - Manter a mensagem breve e impactante.

            c) Buscando Perspectiva:
            - Reconhecer a situação de incerteza, dificuldade ou necessidade de clareza do usuário.
            - Oferecer uma mensagem concisa sobre sabedoria interior, confiança, resiliência ou a importância da intuição.
            - Evitar soluções diretas e focar em oferecer uma nova maneira de pensar sobre a situação.

            3) Tom e Estilo:
            * Manter um tom empático, atencioso e respeitoso em todas as interações.
            * Utilizar linguagem clara, concisa e apropriada ao contexto emocional identificado.
            * Evitar jargões psicológicos complexos.
            * Responder de forma direta e focada na necessidade identificada, sem divagações.

            """,
        description="Agente que fornece a resposta",
    )
    entrada_do_agente_motivador = jornada
    # Executa o agente
    mensagem = call_agent(motivador, entrada_do_agente_motivador)
    return mensagem


In [29]:
    # --- Obter a frase do Usuário ---
    necessidade = input("❓ Por favor, descreva em uma frase como está se sentindo, ou expresse seu humor, ou objetivos: ")
    # Inserir lógica do sistema de agentes ################################################
    if not necessidade:
      print("❌ Você precisa fornecer uma frase.")
    else:
      print("Aguarde! estamos preparando sua mensagem: ")



    # Usando agente 1
      print("\n...🔍 Identificando sua necessidade...")
    jornada = agente_identificador(necessidade)
    print("\n...📝 Resultado do agente 1 (Identificador)  ---\n")
    display(to_markdown(jornada))
    print("------------------------------------------------")

    # Usando agente 2
    print("📝 Gerando mensagem motivacional...\n")
    mensagem = agente_motivador(jornada)
    print("\n...📝 Resultado do agente 2 (Motivador) ---\n")
    display(to_markdown(mensagem))
    print("------------------------------------------------")

    print("\n✅ Mensagem gerada com sucesso!\n")
    print("------------------------------------------------")








❓ Por favor, descreva em uma frase como está se sentindo, ou expresse seu humor, ou objetivos: matando um leão por dia
Aguarde! estamos preparando sua mensagem: 

...🔍 Identificando sua necessidade...

...📝 Resultado do agente 1 (Identificador)  ---



> Buscando Inspiração


------------------------------------------------
📝 Gerando mensagem motivacional...


...📝 Resultado do agente 2 (Motivador) ---



> "Acredite que você pode e já estará no meio do caminho." – Theodore Roosevelt.
> 


------------------------------------------------

✅ Mensagem gerada com sucesso!

------------------------------------------------
